In [1]:
import pandas as pd
import dash_leaflet as dl
import plotly.express as px
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

# CRUD helper
from animal_shelter import AnimalShelter

In [2]:
USER = "root"
PASS = "iZHac1gYyP"
HOST = "nv-desktop-services.apporto.com"
PORT = 31373

# Instantiate CRUD wrapper
shelter = AnimalShelter(
    user=USER,
    password=PASS,
    host=HOST,
    port=PORT,
    db_name="AAC",
    col_name="animals"
)

print("Ping OK? ", shelter.client.admin.command("ping")["ok"])

Ping OK?  1.0


In [3]:
# Pull every document (empty filter) used for the initial view
df = pd.DataFrame.from_records(shelter.read({}))

# Drop Mongo’s ObjectId column to keep Dash DataTable clean
if "_id" in df.columns:
    df = df.drop(columns="_id")

print("Rows:", len(df), "| Columns:", len(df.columns))
df.head()

# ---------- Rescue-type criteria ----------
CRITERIA = {
    "water": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland"
        ]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "mountain": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "disaster": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    # returns every record
    "reset": {}
}

Rows: 10001 | Columns: 16


In [4]:
# ---------------- Dash Layout ----------------
app = JupyterDash(__name__)

app.layout = html.Div([
    # ---------- branding ----------
    html.H1("Grazioso Salvare – AAC Dashboard"),
    html.H2("Author: Chris Davidson"),
    # linked logo (place grazioso_logo.png inside an assets/ folder)
    html.A(
        html.Img(src="/assets/Grazioso Salvare Logo.png", height="60px"),
        href="https://www.snhu.edu",
        target="_blank"
    ),
    html.Hr(),

    # ---------- rescue-type filter ----------
    dcc.RadioItems(
        id="rescue-filter",
        options=[
            {"label": "Water Rescue",    "value": "water"},
            {"label": "Mountain Rescue", "value": "mountain"},
            {"label": "Disaster Rescue", "value": "disaster"},
            {"label": "Reset",           "value": "reset"},
        ],
        value="reset",
        inline=True,
        style={"margin": "10px 0"}
    ),

    # ---------- DataTable ----------
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True}
                 for c in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
    ),

    html.Br(), html.Hr(),

    # ---------- Map & charts ----------
    html.Div(id="map-id", className="col s12 m6"),
    dcc.Graph(id="outcome-pie"),
    dcc.Graph(id="age-scatter"),
])

In [5]:
# ----------- back-end Mongo filtering ----------
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "selected_rows"),
    Input("rescue-filter", "value")
)
def update_table(filter_value):
    """Run the appropriate Mongo query and feed the DataTable."""
    query = CRITERIA.get(filter_value, {})
    dff = pd.DataFrame.from_records(shelter.read(query))

    if "_id" in dff.columns:
        dff = dff.drop(columns="_id")

    # If no rows match, return empty list so other widgets show 'no data'
    return dff.to_dict("records"), ([0] if not dff.empty else [])

# ----------- visuals that rely on the (possibly-filtered) table ----------
@app.callback(
    Output("map-id", "children"),
    Output("outcome-pie", "figure"),
    Output("age-scatter", "figure"),
    Input("datatable-id", "derived_virtual_data"),
    Input("datatable-id", "derived_virtual_selected_rows")
)
def update_visuals(view_data, selected):
    # table empty
    if view_data is None or len(view_data) == 0:
        return [], {}, {}

    dff = pd.DataFrame(view_data)
    row = selected[0] if selected else 0

    # ----- Map marker -----
    lat = dff.iloc[row]["location_lat"]
    lon = dff.iloc[row]["location_long"]
    map_children = [
        dl.Map(style={"width": "750px", "height": "500px"},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]["breed"]),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[row]["name"])
                ])
            ])
        ])
    ]

    # ----- Outcome distribution pie -----
    pie_fig = px.pie(
        dff,
        names="outcome_type",
        title="Outcome Distribution (current table view)",
        hole=0.3
    )

    # ----- Age vs outcome scatter -----
    scatter_fig = px.scatter(
        dff,
        x="age_upon_outcome_in_weeks",
        y="animal_id",
        color="outcome_type",
        title="Age (weeks) vs. Outcome",
        labels={"animal_id": "record"}
    )

    return map_children, pie_fig, scatter_fig

In [6]:
app.run_server(port=8050)

Dash app running on http://127.0.0.1:8050/
